In [1]:
import knaps_cbc, knaps_gurobi
Cls_considered = [knaps_cbc.knapsack_cbc, knaps_gurobi.knapsack_gurobi]
slv_names = ["CBC","Gurobi"]
# If you have only one of solvers installed, edit the above accordingly

## Problem data

In [2]:
# The contents of frs_input.py, generated by frs_random2code.py from random problem data in frs_random.xlsx
# Maximization objectives were converted to minimization by making the coefficients negative.
crit_names = ['Revenues','Carbon storage','Habitat availability']
regime_names = ['Business as usual','Set aside','Extended rotation','No thinning']
areas = [56,53,57,97,52,60,51,87,50,60,90,51,52,55,55,84,64,66,79,81]
coeffs = [[[-29120. ,    -0. ,-24523. ,-22847. ],  [-30634. ,    -0. ,-27608. ,-32095. ],  [-37734. ,    -0. ,-30380. ,-32645. ],  [-49955. ,    -0. ,-45168. ,-43082. ],  [-40300. ,    -0. ,-37198. ,-40946. ],  [-39480. ,    -0. ,-34548. ,-33913. ],  [-38097. ,    -0. ,-31817. ,-38195. ],  [-44544. ,    -0. ,-37742. ,-38729. ],  [-39150. ,    -0. ,-34978. ,-38642. ],  [-38340. ,    -0. ,-31235. ,-33902. ],  [-52380. ,    -0. ,-44576. ,-45175. ],  [-33150. ,    -0. ,-29154. ,-29055. ],  [-37544. ,    -0. ,-33577. ,-29053. ],  [-35035. ,    -0. ,-32713. ,-29798. ],  [-40865. ,    -0. ,-33640. ,-35165. ],  [-55104. ,    -0. ,-45989. ,-43288. ],  [-49920. ,    -0. ,-40279. ,-37950. ],  [-42636. ,    -0. ,-36832. ,-44305. ],  [-50639. ,    -0. ,-45288. ,-50725. ],  [-43254. ,    -0. ,-38922. ,-34498. ]], [[ -1400. , -2128. , -1422. , -1419. ],  [ -1590. , -2014. , -1590. , -1541. ],  [ -1197. , -1881. , -1197. , -1302. ],  [ -2231. , -2910. , -2158. , -2396. ],  [ -1300. , -1976. , -1359. , -1324. ],  [ -1200. , -1920. , -1315. , -1180. ],  [ -1020. , -1581. , -1000. , -1094. ],  [ -2610. , -2784. , -2670. , -2658. ],  [ -1400. , -2000. , -1360. , -1499. ],  [ -1680. , -2400. , -1651. , -1768. ],  [ -1890. , -2700. , -1922. , -1955. ],  [ -1275. , -2040. , -1315. , -1398. ],  [ -1196. , -1820. , -1169. , -1182. ],  [ -1375. , -1760. , -1350. , -1442. ],  [ -1320. , -1815. , -1261. , -1465. ],  [ -2520. , -3108. , -2537. , -2783. ],  [ -1408. , -2560. , -1500. , -1353. ],  [ -1584. , -2376. , -1551. , -1791. ],  [ -1659. , -3081. , -1621. , -1775. ],  [ -1701. , -2430. , -1680. , -1927. ]], [[   -24.9,   -39.2,   -24.3,   -22.7],  [   -23.9,   -42.4,   -31.1,   -23.9],  [   -20. ,   -39.9,   -28.6,   -23.8],  [   -39.4,   -67.9,   -45.5,   -43.1],  [   -26.5,   -41.6,   -25.5,   -26.9],  [   -28.7,   -48. ,   -31.8,   -25.3],  [   -20.5,   -35.7,   -23.7,   -20.1],  [   -44.4,   -69.6,   -50.1,   -38.5],  [   -20.1,   -40. ,   -29.3,   -20.4],  [   -25.1,   -42. ,   -29.1,   -25. ],  [   -39.7,   -63. ,   -40.3,   -37.9],  [   -19.6,   -35.7,   -24.7,   -22. ],  [   -22. ,   -41.6,   -25. ,   -23.2],  [   -26.9,   -44. ,   -26.8,   -24. ],  [   -27.6,   -44. ,   -32.1,   -27.5],  [   -30.2,   -58.8,   -42.9,   -35.7],  [   -25.4,   -44.8,   -27.1,   -27.9],  [   -27.2,   -46.2,   -33.9,   -29.7],  [   -34.2,   -55.3,   -36.2,   -33.4],  [   -37.1,   -64.8,   -44.8,   -38.5]]]

In [3]:
# m - number of classes (forest stands) = number of rows of the coefficients matrix
# n - size of each class (treatment regimes assigned to each stand) = number of columns of the coefficients matrix 
m, n = (len(areas), len(regime_names)) 
total_area = sum(areas)
areas = [[ai] for ai in areas] # making the column vector for using in column constraints

In [4]:
print("Criteria:", crit_names)
print("Management regimes:", regime_names)
print("Number of forest stands:", m)
print("Total area:", total_area)

Criteria: ['Revenues', 'Carbon storage', 'Habitat availability']
Management regimes: ['Business as usual', 'Set aside', 'Extended rotation', 'No thinning']
Number of forest stands: 20
Total area: 1300


## Creating a problem object for each of the considered solvers

In [ ]:
## Initialize the problem for each solver
K_problems = [
                cls(
                    # Shape of the variables array is defined first
                    var_shape = (m,n), 
                    # Function to convert minimized objective values into maximized 
                    # values for output
                    obj2out = lambda dct: {ki: -vi for ki, vi in dct.items()},
                    mute = True # Do not print output of Gurobi solver
                ) 
                  for cls in Cls_considered
             ]
K_main = K_problems[0]

In [6]:
# Fill in each problem object with data
for Ki in K_problems:
    # Objective functions
    for ni, ci in zip(crit_names, coeffs):
        Ki.add_obj(ci, name = ni)
    # Column constraints
    # - no single regime can be assigned to more than 80% of forest area
    Ki.add_col_constrs(areas, 0.8*total_area)
    # - at least 30% of forest area should be set aside
    Ki.add_col_constrs(areas, 0.3*total_area, sense = ">=", col_ids = [1])
    

## Calculating ranges of Pareto frontier


In [7]:
for Ki, sni in zip(K_problems,slv_names):
    Ki.eff_range()
    print(f"*** Solver {sni}: Pareto range ***")
    print("Ideal objective vector:\n",Ki.out_ideal)
    print("Nadir objective vector:\n",Ki.out_nadir)    

*** Solver CBC: Pareto range ***
Ideal objective vector:
 {'Revenues': 617502.0, 'Carbon storage': 44331.0, 'Habitat availability': 913.7}
Nadir objective vector:
 {'Revenues': 124112.0, 'Carbon storage': 35038.99999999999, 'Habitat availability': 673.3000000000001}
*** Solver Gurobi: Pareto range ***
Ideal objective vector:
 {'Revenues': 617502.0, 'Carbon storage': 44331.0, 'Habitat availability': 913.7}
Nadir objective vector:
 {'Revenues': 124112.0, 'Carbon storage': 35038.99999999999, 'Habitat availability': 673.3000000000001}


## Deriving Pareto optimal solutions using ASF
The ideal point plays the role of the reference point.

In [8]:
# Consider two variants of weight vectors
for wi, nwi in [
    [ [1,1,1], "equal priority of objectives" ], 
    [ [5,1,1], "emphasizing revenues" ] 
        ]:
    print(f"\n*** Derived solution for {nwi} ***")
    # Derive a Pareto optimum by each of the considered solver
    results = [Ki.solve_asf(w=wi) for Ki in K_problems]
    # Output results
    print(f"-- Solution derived by {slv_names[0]}")
    print("Objective values:",results[0]["out"])
    print("... as percentage between nadir and ideal:",results[0]["out %"])
    if len(K_problems) > 1:
        print(f"-- Compare with the solution by {slv_names[1]}")
        print("Objective values:",results[1]["out"])
        print("... as percentage between nadir and ideal:",results[1]["out %"])
    print("Assignment of management regimes to stands:")
    for i, ri in enumerate(K_main.bin2int(results[0]["x"])):
        print(f"{i}:", regime_names[ri])    


*** Derived solution for equal priority of objectives ***
-- Solution derived by CBC
Objective values: {'Carbon storage': 40192.00000000001, 'Revenues': 392245.0, 'Habitat availability': 803.8}
... as percentage between nadir and ideal: {'Carbon storage': 55.45630650021535, 'Revenues': 54.34504144794179, 'Habitat availability': 54.28452579034938}
-- Compare with the solution by Gurobi
Objective values: {'Carbon storage': 40192.00000000001, 'Revenues': 392245.00000000006, 'Habitat availability': 803.8000000000015}
... as percentage between nadir and ideal: {'Carbon storage': 55.45630650021535, 'Revenues': 54.3450414479418, 'Habitat availability': 54.28452579035004}
Assignment of management regimes to stands:
0: Set aside
1: Extended rotation
2: Set aside
3: Set aside
4: No thinning
5: Set aside
6: Business as usual
7: Extended rotation
8: Extended rotation
9: Set aside
10: Business as usual
11: Set aside
12: Set aside
13: Business as usual
14: No thinning
15: Extended rotation
16: Set 

In [9]:
# Assume that the second solution is more satisfactory, but we want to reach the 40K goal for Carbon storage
K_main._obj_bounds({'Carbon storage': -40000}) # Note the internal representation in terms of minimization
res_money_carbon = K_main.solve_asf(w = [5,1,1])
print("\n*** Derived solution for emphasizing revenues & bounded Carbon storage ***")
print("Objective values:",res_money_carbon["out"])
print("... as percentage between nadir and ideal:",res_money_carbon["out %"])
print("Assignment of management regimes to stands:")
for i, ri in enumerate(K_main.bin2int(res_money_carbon["x"])):
    print(f"{i}:", regime_names[ri])


*** Derived solution for emphasizing revenues & bounded Carbon storage ***
Objective values: {'Carbon storage': 40000.0, 'Revenues': 448524.0, 'Habitat availability': 736.5999999999999}
... as percentage between nadir and ideal: {'Carbon storage': 53.39001291433495, 'Revenues': 65.75163663633231, 'Habitat availability': 26.331114808652185}
Assignment of management regimes to stands:
0: Set aside
1: Business as usual
2: Set aside
3: No thinning
4: Set aside
5: Set aside
6: No thinning
7: Business as usual
8: No thinning
9: Set aside
10: Business as usual
11: Set aside
12: Set aside
13: Business as usual
14: No thinning
15: No thinning
16: Set aside
17: No thinning
18: Set aside
19: Business as usual
